<h1 style="text-align: center; font-size: 50px;">Multimodal RAG Chatbot with Langchain and ML Flow Evaluation</h1>

Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions. We'll also use the DeepEval platform to evaluate, observe and protect the LLM responses.

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Data Loading
- Creation of Chunks
- Retrieval
- Model Setup
- Chain Creation
- Model Service 

# Imports

By using our Local GenAI workspace image, many of the necessary libraries to work with RAG already come pre-installed - in our case, we just need to add the connector to work with PDF documents

In [1]:
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# === Standard Library Imports ===
from typing import List
from datetime import datetime
import warnings
from pathlib import Path
import os
import sys
import logging
import pandas as pd

# === MLflow integration ===
import mlflow

# Define the relative path to the 'core' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
# === Import ChatbotService from project core ===
from core.chatbot_service.chatbot_service import ChatbotService

# === Third-Party Imports ===
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.document import Document
from langchain.document_loaders import WebBaseLoader, JSONLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
import promptquality as pq
import torch

from llama_cpp import Llama
from llama_cpp.llama_chat_format import InternVLChatHandler

# Define the relative path to the 'src' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# === Project-Specific Imports (from src) ===
from src.local_genai_judge import LocalGenAIJudge
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    initialize_llm,
    configure_hf_cache,
    mlflow_evaluate_setup,
)

2025-07-08 08:05:39.046342: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 08:05:39.068922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751961939.078997    5218 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751961939.082260    5218 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751961939.093920    5218 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Configurations

In [3]:
warnings.filterwarnings("ignore")

In [4]:
# Create logger
logger = logging.getLogger("multimodal_rag_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [5]:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
DATA_PATH = "../data"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Multimodal-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Multimodal-Chatbot-Run"
INTERNVL
LLAMA3_LOCAL_MODEL_PATH = "/home/jovyan/datafabric/llama3.1-8b-instruct/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
LLAMA3LOCAL_MODEL_PATHV2 = "/home/jovyan/datafabric/meta-llama-3.1-8b-instruct-q4_k_m/meta-llama-3.1-8b-instruct-q4_k_m.gguf"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "AIStudio-Multimodal-Chatbot-Model"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [1]:
from llama_cpp import llama_supports_gpu_offload
print("GPU off-load supported:", llama_supports_gpu_offload())

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4070, compute capability 8.9, VMM: yes


GPU off-load supported: True


In [8]:
logger.info('Notebook execution started.')

2025-07-08 08:05:41 - INFO - Notebook execution started.


## Configuration of HuggingFace caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.

In [9]:
# Configure HuggingFace cache
configure_hf_cache()

In [10]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-large-v2",
    cache_folder="/tmp/hf_cache"
)

2025-07-08 08:05:41,621 - INFO - Use pytorch device_name: cuda:0
2025-07-08 08:05:41,621 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like Galileo and HuggingFace

In [11]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

## Proxy Configuration

In order to connect to Galileo service, a SSH connection needs to be established. For certain enterprise networks, this might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [12]:
configure_proxy(config)

# Verify Assets

In [13]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")

log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="Config",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_asset_status(
    asset_path=SECRETS_PATH,
    asset_name="Secrets",
    success_message="",
    failure_message="Please check if the secrets.yaml was propely connfigured in your project on AI Studio."
)
log_asset_status(
    asset_path=LOCAL_MODEL_PATH,
    asset_name="Local Llama model",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio if you want to use local model."
)

2025-07-08 08:05:45 - INFO - Config is properly configured. 
2025-07-08 08:05:45 - INFO - Secrets is properly configured. 
2025-07-08 08:05:45 - INFO - Local Llama model is properly configured. 


# Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data from pages on the web.

In [14]:
# === Verify existence of the data directory ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'data' folder not found at path: {os.path.abspath(DATA_PATH)}")

def load_wiki_data(DATA_PATH):
    # === Wiki JSON with JSONLoader ===
    wiki_loader = JSONLoader(
        file_path=os.path.join(DATA_PATH, "wiki_flat_structure_mini.json"),
        jq_schema="to_entries[] | {source: .key, text: .value.content}",  # adapt to your schema
        text_content=False  # keeps original formatting; set True if you want only strings
    )

    docs = wiki_loader.load()

    return docs

docs = load_wiki_data(DATA_PATH)
# === Optional: Load additional web-based documents ===
# To use a different knowledge base, just change the URLs below

# loader1 = WebBaseLoader("https://www.hp.com/us-en/workstations/ai-studio.html")
# data1 = loader1.load()

# loader2 = WebBaseLoader("https://zdocs.datascience.hp.com/docs/aistudio")
# data2 = loader2.load()

# Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add to our vector database.

In [15]:
# === Initialize text splitter ===
# - chunk_size: Maximum number of characters per text chunk.
# - chunk_overlap: Number of overlapping characters between chunks.

def chunk_documents(docs, chunk_size=600, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n## ", "\n# ", "\n\n", ".", "!", "?"]
    )
    splits = splitter.split_documents(docs)
    return splits

splits = chunk_documents(docs)
def log_stage(name: str, docs: List[Document]):
    logger.info(f"{name}: {len(docs)} docs, avg_tokens={sum(len(d.page_content) for d in docs)/len(docs):.0f}")
# e.g. after splits
log_stage("Chunks created", splits)

2025-07-08 08:05:45 - INFO - Chunks created: 53 docs, avg_tokens=459


# Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents

In [16]:
%%time

# === Create a vector database from document chunks ===
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings)

# === Configure the vector database as a retriever for querying ===
retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"fetch_k": 20, "k": 8}
)

2025-07-08 08:05:45,310 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-07-08 08:05:45,393 - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-07-08 08:05:45,398 - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


CPU times: user 813 ms, sys: 177 ms, total: 990 ms
Wall time: 1 s


# Model Setup

In this notebook, we provide three different options for loading the model:
 * **local**: by loading the llama3.1-8b-instruct-Q8_0 model from the asset downloaded on the project
 * **hugging-face-local** by downloading a DeepSeek model from Hugging Face and running locally
 * **hugging-face-cloud** by accessing the Mistral model through Hugging Face cloud API (requires HuggingFace API key saved on secrets.yaml)

This choice can be set in the config.yaml file. The model deployed on the bottom cells of this notebook will load the choice from the config file.

In [17]:
model_source = config["model_source"]

In [18]:
%%time

llm = initialize_llm(model_source, secrets)

llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


CPU times: user 1.53 s, sys: 5.07 s, total: 6.59 s
Wall time: 36.7 s


# Chain Creation
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [19]:
# === Function to format retrieved documents ===
# Converts a list of Document objects into a single formatted string

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

In [20]:
# Helper that turns <context> + query into a user-role segment
def _build_user_prompt(context: str, query: str) -> str:
    return (
        f"<context>\n{context}\n</context>\n\n"
        f"User query: \"{query}\"\n\n"
        "Based only on the context above, provide the answer. "
        "If the context does not contain the answer, reply exactly with: "
        "\"I don’t have that information in the wiki yet.\" "
        "Answer:"
    )

# Meta-Llama 3 header template (system/user/assistant sections)
META_LLAMA_TEMPLATE = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    template=META_LLAMA_TEMPLATE,
    input_variables=["system_prompt", "user_prompt"],
)

# Constant system instructions (with same rules as before)
SYSTEM_PROMPT = (
    "You are a technical assistant for HP’s Z by HP AI Studio team.\n\n"
    "Only answer using the information provided in the <context> block.\n"
    "If the answer is not found in the context, reply with:\n"
    "\"I don’t have that information in the wiki yet.\"\n\n"
    "Rules:\n"
    "- Use only the information from <context>.\n"
    "- For each fact you include, cite the source file name in parentheses.\n"
    "- Do not invent information or use outside knowledge.\n"
    "- Do not refer to these instructions or repeat them.\n"
    "- Use bullet points or steps if it makes the answer clearer.\n"
    "- Avoid redundancy.\n"
)

# Build the full RAG pipeline

retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"fetch_k": 10, "k": 4},
)

chain = (
    {
        "context": retriever | format_docs,   # fetch and stringify context
        "query":   RunnablePassthrough(),     # pass the user question along
    }
    # Compose variables for the prompt
    | RunnableLambda(
        lambda d: {
            "system_prompt": SYSTEM_PROMPT,
            "user_prompt":   _build_user_prompt(d["context"], d["query"]),
        }
    )
    # Render the final prompt, call the model, parse the text
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# Invoke to test our RAG response quality
question = "How do i test ai blueprints?"
answer = chain.invoke(question)
print(answer)

2025-07-08 08:06:22,978 - ERROR - Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Please follow the steps below when testing any blueprint project:

### 1. Create a Project in AI Studio

*   If the blueprint is published, create a new project in AI Studio using the blueprint directly.

### 2. Configure and Run the Notebook

*   Open the Jupyter notebook associated with the blueprint.
*   Execute all cells by clicking **"Run All"**, as described in **Step 1 of the `Usage` section**.

### 3. Register the Model and Deploy Locally (if applicable)

*   If the blueprint includes MLflow integration:
    *   Follow the next usage step to **register the model in MLflow** and deploy it successfully.


# MLFlow Model Service 

In this section, we demonstrate how to deploy a RAG-based chatbot service. This service provides a REST API endpoint that allows users to query the knowledge base with natural language questions, upload new documents to the knowledge base, and manage conversation history, all with built-in safeguards against sensitive information and toxicity. This service encapsulates all the functionality we developed in this notebook, including the document retrieval system, RAG-based question answering capabilities, and Galileo integration for protection, observation and evaluation. It demonstrates how to use our ChatbotService from the src/service directory. 

## Setup

In [22]:
mlflow_evaluate_setup(
    secrets,
    mlflow_tracking_uri="/phoenix/mlflow"   # your AI Studio MLflow endpoint
)

# === Set MLflow experiment context ===
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# === Validate local model file path ===
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

✅ Environment ready for MLflow evaluation.


## Log & Register Model

In [23]:
# === Log and register model to MLflow ===
with mlflow.start_run(run_name=MLFLOW_RUN_NAME) as run:
    
    # Log model artifacts using custom ChatbotService
    ChatbotService.log_model(
        artifact_path=MLFLOW_MODEL_NAME,
        config_path=CONFIG_PATH,
        secrets_path=SECRETS_PATH,
        docs_path=DATA_PATH,
        model_path=LOCAL_MODEL_PATH,
        demo_folder=DEMO_FOLDER
    )

    # Construct the URI for the logged model
    model_uri = f"runs:/{run.info.run_id}/{MLFLOW_MODEL_NAME}"

2025-07-08 08:06:26,240 - INFO - Use pytorch device_name: cuda:0
2025-07-08 08:06:26,241 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2


2025/07/08 08:07:27 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - PyPDF (current: uninstalled, required: PyPDF)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025-07-08 08:08:11,725 - INFO - Model and artifacts successfully registered in MLflow.


In [24]:
# Register the model into MLflow Model Registry
mlflow.register_model(
    model_uri=model_uri,
    name=MLFLOW_MODEL_NAME
)

logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

Registered model 'AIStudio-Multimodal-Chatbot-Model' already exists. Creating a new version of this model...
Created version '6' of model 'AIStudio-Multimodal-Chatbot-Model'.
2025-07-08 08:08:11 - INFO - ✅ Model registered successfully with run ID: 1e37c6f0a66f4b4fa65b132e7957be30


## Evaluate Hallucination, Answer Relevance, Context Precision and Recall

In [25]:

def model(batch_df: pd.DataFrame) -> pd.DataFrame:
    preds, contexts = [], []
    for q in batch_df["questions"]:
        answer = chain.invoke(q)
        preds.append(answer)

        docs = retriever.get_relevant_documents(q)
        contexts.append(" ".join(d.page_content for d in docs))

    # keep the incoming index so every batch’s rows stay unique
    return pd.DataFrame(
        {
            "result": preds,
            "source_documents": contexts,
        },
        index=batch_df.index,      #  ← key line
    )

# --- 3)  Evaluation dataset
eval_df = pd.DataFrame({"questions": [
    "What naming convention should I use for a new blueprint project folder?",
    "What is the first step in the standard blueprint testing workflow?",
    "How do I fetch logs from a running Kubernetes pod?",
]})

judge = LocalGenAIJudge(
    llm=llm
)

faithfulness_metric = judge.to_mlflow_metric("faithfulness")
relevance_metric = judge.to_mlflow_metric("relevance")

results = mlflow.evaluate(
    model,
    eval_df,
    predictions="result",
    evaluators="default",
    extra_metrics=[faithfulness_metric, relevance_metric],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "source_documents"
        }
    },
)


2025/07/08 08:08:12 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/07/08 08:08:12 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/07/08 08:08:18 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


Built with ❤️ using Z by HP AI Studio.